<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers_Entrenament_Complert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Entrenament complert**

---
Aquest procediment presenta com entrenar un model utilitzant un bucle d'entrenament personalitzat:​

1. **Preprocessament de dades**: Similar a l'anterior, però amb passos addicionals com eliminar columnes no necessàries i establir el format de les dades per retornar tensors de PyTorch.​
huggingface.co

1. **Creació de DataLoaders**: S'utilitzen DataLoader de PyTorch per crear iteradors sobre els conjunts de dades d'entrenament i validació.​

1. **Creació del model**: Es carrega el model amb AutoModelForSequenceClassification.​

1. **Entrenament**: S'implementa un bucle d'entrenament manual, on es defineixen explícitament els passos d'entrenament i validació, incloent la pèrdua i l'optimitzador.​

Aquest enfocament ofereix més flexibilitat i control sobre el procés d'entrenament, permetent personalitzacions específiques segons les necessitats del projecte.


# **Preprocessament i preparació de les dades**

---
1. **Carrega del dataset**:


In [ ]:
!pip install datasets
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")

**2. Tokenització:**

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


3. **Postprocessament per adaptar el dataset al model:**

  * Elimina columnes no necessàries (sentence1, sentence2, idx).

  * Renombra label a labels (el model espera aquesta clau).

  * Estableix el format del dataset perquè retorni tensors de PyTorch:

In [3]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")


4. **Creació de DataLoaders**:

Els DataLoaders són una eina que proporciona PyTorch (i altres frameworks) per facilitar el maneig i subministrament de dades a un model durant l'entrenament o la validació.

Es pot veure com un contenidor intel·ligent que:

  * Accedeix al conjunt de dades,

  * N’agafa mostres en lots (batches),

  * Les barreja (si cal),

  * I les envia al model d’una forma eficient i ordenada.

In [4]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator)


# **Creació i preparació del model**

---


5. **Càrrega del model amb capa de classificació:**

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


6. **Mou el model al dispositiu correcte (GPU si disponible):**

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


# **Configuració de l'optimitzador i scheduler**

---

7. **Optimitzador AdamW:**

In [8]:
# from transformers import AdamW  dona error
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)


8. **Scheduler de taxa d’aprenentatge lineal:**

In [9]:
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


# **Bucle d’entrenament manual**

---


10. **Entrenament amb tqdm per mostrar el progrés:**

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


# **Bucle d’avaluació**

---


11. **Càrrega i ús de la mètrica:**

In [ ]:
!pip install evaluate --quiet
import evaluate
metric = evaluate.load("glue", "mrpc")
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    predictions = torch.argmax(outputs.logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())
